In [1]:
import cellCnn
import importlib

import numpy as np
### (from: https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/NK_cell_ungated.ipynb ) AND https://github.com/eiriniar/CellCnn/blob/0413a9f49fe0831c8fe3280957fb341f9e028d2d/cellCnn/examples/PBMC.ipynb
import pandas as pd
from sklearn.metrics import roc_auc_score
from sklearn.utils import shuffle
import seaborn as sns
import matplotlib.pyplot as plt

from cellCnn.ms.utils.helpers import get_min_mean_from_clusters, get_chunks
from cellCnn.utils import mkdir_p
from cellCnn.plotting import plot_results
from cellCnn.ms.utils.helpers import get_chunks
from cellCnn.ms.utils.helpers import print_regression_model_stats
from cellCnn.plotting import plot_results
from cellCnn.utils import mkdir_p
from cellCnn.utils import save_results
from cellCnn.ms.utils.helpers import get_fitted_model
from cellCnn.ms.utils.helpers import split_test_valid_train
from cellCnn.ms.utils.helpers import calc_frequencies, get_chunks_from_df

import os
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

2022-03-24 10:10:24.337624: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-24 10:10:24.337672: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-24 10:10:24.338283: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


In [2]:
#%reset

In [3]:
##### state vars
cytokines = ['CCR2', 'CCR4', 'CCR6', 'CCR7', 'CXCR4', 'CXCR5', 'CD103', 'CD14', 'CD20', 'CD25', 'CD27', 'CD28', 'CD3',
             'CD4', 'CD45RA', 'CD45RO', 'CD56', 'CD57', 'CD69', 'CD8', 'TCRgd', 'PD.1', 'GM.CSF', 'IFN.g', 'IL.10',
             'IL.13', 'IL.17A', 'IL.2', 'IL.21', 'IL.22', 'IL.3', 'IL.4', 'IL.6', 'IL.9', 'TNF.a']
infile = 'cohort_denoised_clustered_diagnosis_patients.csv'
indir = 'data/input.nosync'
outdir = 'out_ms_default'
rand_seed = 123
train_perc = 0.7
test_perc = 0.3
batch_size_pheno = 840 # deprecated  # so a size of 8425 is about equally sized in batches
batch_size_cd4 = 550  # deprecated #so a size of 550 gets me 16 batches for cd4
## information from ms_data project
cluster_to_celltype_dict = {0: 'b', 1: 'cd4', 3: 'nkt', 8: 'cd8', 10: 'nk', 11: 'my', 16: 'dg'}
outfolder = 'mtl_1'
mkdir_p(outfolder)

In [4]:
np.random.seed(rand_seed)
mkdir_p(outdir)
df = pd.read_csv(os.path.join(indir, infile), index_col=0)
df = df.drop_duplicates()  ### reduces overfitting at cost of fewer data
df.shape
##### no duplicates in

(16889, 38)

In [5]:
# pitch: key = gate_source, value = dict{diagnosis: df OR freq?}
rrms_df = df[df['diagnosis'] == 'RRMS']
rrms_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                    rrms_df.groupby('gate_source')}
nindc_df = df[df['diagnosis'] == 'NINDC']
nindc_patients2df = {id: patient_df.drop(columns=['diagnosis', 'gate_source']) for id, patient_df in
                     nindc_df.groupby('gate_source')}

In [6]:
#### here we could see freq differences across the 2 groups
print('Frequencies: ')
rrms_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df in
                      rrms_patients2df.items()}
nindc_patients_freq = {id: calc_frequencies(patient_df, cluster_to_celltype_dict, return_list=True) for id, patient_df
                       in nindc_patients2df.items()}
print('DONE')
# we got 31 patients each

Frequencies: 
DONE


In [7]:
patients_clusters_conf_table = pd.crosstab(df['gate_source'], df['cluster'])
sns.heatmap(patients_clusters_conf_table, annot=False, vmax=100)
plt.show()
#plt.savefig('images/patient_vs_cluster_conf_table.png')
patients_clusters_conf_table

/var/folders/p1/5d38cf1d6tvdyny38zb_66540000gn/T/ipykernel_54229/1790410289.py:3: UserWarning: Matplotlib is currently using agg, which is a non-GUI backend, so cannot show the figure.
  plt.show()


cluster,0,1,3,8,10,11,16
gate_source,,,,,,,
3.0,60,119,6,30,5,5,0
4.0,15,46,0,45,2,1,13
5.0,33,131,0,70,2,2,4
8.0,24,122,0,31,10,11,4
9.0,72,270,2,137,11,10,4
...,...,...,...,...,...,...,...
128.0,4,6,4,6,1,6,0
130.0,24,82,2,56,8,4,0
131.0,20,48,2,14,13,7,0


In [8]:
# todo ich muss eigentlich die frequency pro batch berechnen...
importlib.reload(cellCnn.ms.utils.helpers)
from cellCnn.ms.utils.helpers import *

# batch_sizes = [40,80,120, 160]
batch_size_dict = dict()
batch_sizes = [120]
for batch_size in batch_sizes:
    ### desease states 1 = RRMS and 0 = NINDC
    selection_pool_rrms_cd8 = get_chunks_from_df(rrms_patients2df,
                           freq_df=rrms_patients_freq,
                           desease_state=1,
                           batch_size=batch_size)
    selection_pool_nindc_cd8 = get_chunks_from_df(nindc_patients2df,
                           freq_df=nindc_patients_freq,
                           desease_state=0,
                           batch_size=batch_size)

    # make sure list are equally long:
    if len(selection_pool_rrms_cd8) > len(selection_pool_nindc_cd8):
        selection_pool_rrms_cd8 = selection_pool_rrms_cd8[:len(selection_pool_nindc_cd8)]
    elif len(selection_pool_rrms_cd8) < len(selection_pool_nindc_cd8):
        selection_pool_nindc_cd8 = selection_pool_nindc_cd8[:len(selection_pool_rrms_cd8)]

    all_chunks = selection_pool_rrms_cd8 + selection_pool_nindc_cd8
    np.random.shuffle(all_chunks)

    X = [selection[0] for selection in all_chunks]
    freqs = [selection[1] for selection in all_chunks]
    Y = [selection[2] for selection in all_chunks]
    batch_size_dict[batch_size] = (X, freqs, Y)

print('DONE: batches created')

DONE: batches created


In [10]:
### CD8 Freq regression task + MTL
model_container_cd8 = []
stats_dict_reg_cd8 = dict()
outdir_cd8 = 'mtl/class/cd8'
mkdir_p(outdir_cd8)

In [40]:
%reload_ext autoreload
%autoreload 2
%aimport cellCnn
from cellCnn import loss
from cellCnn.loss import *
import cellCnn
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
importlib.reload(cellCnn.loss)
importlib.reload(cellCnn.utils)
importlib.reload(cellCnn)
from cellCnn.loss import *
from cellCnn.ms.utils.helpers import *
import cellCnn
from cellCnn.loss import RevisedUncertaintyLoss

kf_split = 2
for batch_size, values, in batch_size_dict.items():
    # for regression task stratified is wrong since there are no classes
    freq_idx =3
    X, cd8, y = values[0], values[1], values[2]
    cd8 = [series[freq_idx] for series in cd8]

    X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
        X, cd8, y,
        train_perc=train_perc,
        test_perc=test_perc,
        valid_perc=0.5)
    outdir_pheno_reg_cd8 = f'{outdir_cd8}/models/ms_class_cd8_{batch_size}'

    # todo nsubset parameter maybe different
    model = get_fitted_model(X_train, X_valid, [y_train, cd8_train], [cd8_valid, y_valid],
                             nsubset=100,
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=3, learning_rate=None,
                             ncell=200,
                             per_sample=True, regression=False,
                             outdir='test', verbose=False, mtl_tasks=2)
    model_container_cd8.append(model)
print('DONE building models')
print('DONE class with per_sampel true')

CALLING...
in mtl: split inputs
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "model_87"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data_input (InputLayer)        [(None, 200, 35)]    0           []                               
                                                                                                  
 conv1 (Conv1D)                 (None, 200, 35)      1260        ['data_input[0][0]']             
                                                                                                  
 lambda_87 (Lambda)             (None, 35)           0           ['conv1[0][0]']                  
                                                                                               

2022-03-24 20:53:25.315216: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-24 20:53:25.315240: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-24 20:53:25.316489: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs


2022-03-24 20:53:26.363382: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-24 20:53:26.363402: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-24 20:53:26.485086: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-03-24 20:53:26.486262: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-03-24 20:53:26.488623: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: test/stats/tensorboard/model_87_1648151605.3147619/train/plugins/profile/2022_03_24_20_53_26

2022-03-24 20:53:26.490423: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to test/stats/tensorboard/model_87_1648151605.3147619/train/plugins/profile/2022_03_24_20_53_26/Eliass-MacBook-Pro.local.trace.json.gz
2022-03-24 20:53:26.492618: I tensorflow/core/profiler/rpc/client/save_profile.c

CALLING...
in mtl: split inputs
29/29 [==============================] - 0s 5ms/step - loss: 1.8595 - revised_uncertainty_loss_v2_87_accuracy: 0.5600 - revised_uncertainty_loss_v2_87_1_mean_squared_error: 0.1803
CALLING...
in mtl: split inputs
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "model_88"
_______________________________________________________

...

2022-03-24 20:54:36.778627: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-24 20:54:36.778697: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-24 20:54:36.779248: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


CALLING...
in mtl: split inputs
CALLING...
in mtl: split inputs


2022-03-24 20:54:39.093105: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-24 20:54:39.093126: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-24 20:54:39.172305: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2022-03-24 20:54:39.173442: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-03-24 20:54:39.175388: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: test/stats/tensorboard/model_89_1648151676.777947/train/plugins/profile/2022_03_24_20_54_39

2022-03-24 20:54:39.177057: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to test/stats/tensorboard/model_89_1648151676.777947/train/plugins/profile/2022_03_24_20_54_39/Eliass-MacBook-Pro.local.trace.json.gz
2022-03-24 20:54:39.179312: I tensorflow/core/profiler/rpc/client/save_profile.cc:

CALLING...
in mtl: split inputs
29/29 [==============================] - 0s 3ms/step - loss: 2.1758 - revised_uncertainty_loss_v2_89_accuracy: 0.5711 - revised_uncertainty_loss_v2_89_1_mean_squared_error: 0.1775
DONE building models
DONE class with per_sampel true


/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [30]:
######################################################################
accs = []

# what predict does is: Iterating over the best 3 nets and return an array per net with the predictions for all samples!
bs_idx = 1
for i, model in enumerate(model_container_cd8):
    stats_file = open(f"{outdir_cd8}/stats_class_{model.ncell}_{model.nsubset}.txt", "w+")
    stats_file.write(f"Model {i}; Batchsize: {batch_sizes[bs_idx-1]}\n")
    if bs_idx % kf_split == 0: ## when the kf split amount is reaches the next batchsize is applied
        bs_idx = bs_idx +1
    test_pred = model.predict(X_test)
    train_pred = model.predict(X_train)
    valid_pred = model.predict(X_valid)
    #print_regression_model_stats(test_pred, b_test)
    test_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in test_pred[0]]
    train_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in train_pred[0]]
    valid_pred_abs = [1 if pred[0]>pred[1] else 0 for pred in valid_pred[0]]
    acc_test = accuracy_score(y_test,  test_pred_abs)
    acc_train = accuracy_score(y_train,  train_pred_abs)
    acc_valid = accuracy_score(y_valid,  valid_pred_abs)
    accs.append(acc_test)

    stats_file.write('Desease: \n')
    stats_file.write(f'Acc y test {acc_test}\n')
    stats_file.write(f'Acc y train {acc_train}\n')
    stats_file.write(f'Acc y valid {acc_valid}\n')

    mse_test_cd8 = mean_squared_error(cd8_test, test_pred[1])
    mse_train_cd8 = mean_squared_error(cd8_train, train_pred[1])
    mse_valid_cd8 = mean_squared_error(cd8_valid, valid_pred[1])
    mses.append(mse_test_cd8)

    stats_file.write('Cd8: \n')
    stats_file.write(f'MSE y test {mse_test_cd8}\n')
    stats_file.write(f'MSE y train {mse_train_cd8}\n')
    stats_file.write(f'MSE y valid {mse_valid_cd8}\n')
    stats_file.write('\n')
    stats_file.close()
print('DONE')

[autoreload of loss failed: Traceback (most recent call last):
  File "/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 257, in check
    superreload(m, reload, self.old_objects)
  File "/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 480, in superreload
    update_generic(old_obj, new_obj)
  File "/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 377, in update_generic
    update(a, b)
  File "/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 345, in update_class
    update_instances(old, new)
  File "/Users/eliasschreiner/.local/share/virtualenvs/THESIS-UFQOor4v/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 299, in update_instances
    refs 

TypeError: super(type, obj): obj must be an instance or subtype of type

In [39]:
### CD8 Freq regression task + MTL
model_container_cd8_class = []
stats_dict_cd8_class = dict()

In [31]:
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
importlib.reload(cellCnn)
importlib.reload(cellCnn.utils)
from cellCnn.utils import *
from cellCnn.ms.utils.helpers import *
from cellCnn import *


kf_split = 2
for batch_size, values, in batch_size_dict.items():
    # for regression task stratified is wrong since there are no classes
    freq_idx =3
    X, cd8, y = values[0], values[1], values[2]
    cd8 = [series[freq_idx] for series in cd8]

    X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
        X, cd8, y,
        train_perc=train_perc,
        test_perc=test_perc,
        valid_perc=0.5)
    outdir_pheno_reg_cd8 = f'{outdir_cd8}/models/ms_class_cd8_{batch_size}_psFalse'

    # todo nsubset parameter maybe different
    # yet you need to put phenotype data in frist place
    model = get_fitted_model(X_train, X_valid, [y_train, cd8_train], [y_valid, cd8_valid],
                             nsubset=100,
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=10, learning_rate=None,
                             ncell=200,
                             per_sample=False, regression=False,
                             outdir=outdir_pheno_reg_cd8, verbose=False, mtl_tasks=2)
    model_container_cd8_class.append(model)
print('DONE building models')
print('DONE class with per_sampel false')

Build called


ValueError: Could not interpret initializer identifier: <tf.Variable 'revised_uncertainty_loss_56/sigmas_sq_0:0' shape=() dtype=float32, numpy=0.0>

In [12]:
importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
importlib.reload(cellCnn)
importlib.reload(cellCnn.utils)
from cellCnn.utils import *
from cellCnn.ms.utils.helpers import *
from cellCnn import *


kf_split = 2
for batch_size, values, in batch_size_dict.items():
    # for regression task stratified is wrong since there are no classes
    freq_idx =3
    X, cd8, y = values[0], values[1], values[2]
    cd8 = [series[freq_idx] for series in cd8]

    X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
        X, cd8, y,
        train_perc=train_perc,
        test_perc=test_perc,
        valid_perc=0.5)
    outdir_pheno_reg_cd8 = f'{outdir_cd8}/models/ms_class_cd8_{batch_size}_psFalse'

    # todo nsubset parameter maybe different
    # yet you need to put phenotype data in frist place
    model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                             nsubset=100,
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=10, learning_rate=None,
                             ncell=200,
                             per_sample=False, regression=False,
                             outdir=outdir_pheno_reg_cd8, verbose=False, mtl_tasks=1)
    model_container_cd8_2.append(model)
print('DONE building models')
print('DONE class with per_sampel false')



2022-03-24 09:31:11.497542: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-24 09:31:11.497566: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-24 09:31:11.497702: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-03-24 09:31:11.822029: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-24 09:31:11.822048: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-24 09:31:11.822180: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2022-03-24 09:31:11.894731: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2022-03-24 09:31:11.894749: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2022-03-24 09:31:11.894890: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session 

Build called
CALLING...
in mtl: split inputs
sigma 0
<tf.Variable 'revised_uncertainty_loss_4/sigmas_sq_0:0' shape=() dtype=float32>
listed_loss task 0
Tensor("revised_uncertainty_loss_4/loss:0", shape=(), dtype=float32)
loss
Tensor("revised_uncertainty_loss_4/Min:0", shape=(), dtype=float32)
You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model/model_to_dot to work.
Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 data_input (InputLayer)        [(None, 200, 35)]    0           []                               
                                                                                                  
 conv1 (Conv1D)                 (None, 200, 3)       108         ['data_input[0][0]']             
                                

KeyboardInterrupt: 

In [12]:
model_container_cd8_outlier = []

In [11]:

importlib.reload(cellCnn.ms.utils.helpers)
importlib.reload(cellCnn.model)
importlib.reload(cellCnn)
importlib.reload(cellCnn.utils)
from cellCnn.utils import *
from cellCnn.ms.utils.helpers import *
from cellCnn import *


kf_split = 2
for batch_size, values, in batch_size_dict.items():
    # for regression task stratified is wrong since there are no classes
    freq_idx =3
    X, cd8, y = values[0], values[1], values[2]
    cd8 = [series[freq_idx] for series in cd8]

    X_test, X_train, X_valid, cd8_test, cd8_train, cd8_valid, y_test, y_train, y_valid = split_test_train_valid(
        X, cd8, y,
        train_perc=train_perc,
        test_perc=test_perc,
        valid_perc=0.5)
    outdir_pheno_reg_cd8 = f'{outdir_cd8}/models/ms_class_cd8_{batch_size}_psFalse'

    # todo nsubset parameter maybe different
    # yet you need to put phenotype data in frist place
    model = get_fitted_model(X_train, X_valid, y_train, y_valid,
                             nsubset=100,
                             nfilters=[3, 15, 25, 35], coeff_l1=0,
                             max_epochs=100, nrun=10, learning_rate=None,
                             ncell=200, subset_selection='outlier',
                             per_sample=False, regression=False,
                             outdir=outdir_pheno_reg_cd8, verbose=False, mtl_tasks=1)
    model_container_cd8_outlier.append(model)
print('DONE building models')
print('DONE class with per_sampel false')

